ausgeführt 2024-01-31

In [14]:
from acdh_tei_pyutils.tei import TeiReader
from acdh_tei_pyutils.utils import get_xmlid, normalize_string
from tqdm import tqdm
from icecream import ic
from normdata.utils import import_from_normdata

In [2]:
source_file = "https://raw.githubusercontent.com/hermann-bahr/bahr-index/main/tsn/listwork_TSN_1.xml"

In [3]:
doc = TeiReader(source_file)

In [4]:
nsmap = doc.nsmap
bahr_url = "https://hermanbahrtextverzeichnis/"
bahr_domain = "hermanbahrtextverzeichnis"
bahr_col, _ = Collection.objects.get_or_create(name="Bahr Textverzeichnis")

In [ ]:
for x in tqdm(doc.any_xpath(".//tei:bibl[@xml:id]")):
    entity = False
    try:
        hbtv_uri = get_xmlid(x)
    except KeyError:
        continue
    hbtv_url = f"{bahr_url}{hbtv_uri}"
    uri, _ = Uri.objects.get_or_create(uri=hbtv_url, domain=bahr_domain)
    try:
        name = x.xpath("./tei:title[1]/text()", namespaces=nsmap)[0]    
    except IndexError:
        print(hbtv_url)
        continue
    try:
        entity = uri.entity.get_child_entity()
    except:
        entity = Work.objects.create(name=name)
        entity.collection.add(bahr_col)
        uri.entity = entity
        uri.save()

In [ ]:
personwork_type= PersonWorkRelation.objects.get(id=1049)

In [ ]:
for x in tqdm(doc.any_xpath(".//tei:bibl[@xml:id][./tei:author[./@ref]]")):
    hbtv_uri = get_xmlid(x)
    work_uri = f"{bahr_url}{hbtv_uri}"
    author_ref = x.xpath("./tei:author/@ref", namespaces=nsmap)[0]
    author_uri = f"{bahr_url}{author_ref}"
    try:
        author = Uri.objects.get(uri=author_uri).entity.get_child_entity()
    except:
        continue
    work = Uri.objects.get(uri=work_uri).entity.get_child_entity()
    PersonWork.objects.get_or_create(related_person=author, related_work=work, relation_type=personwork_type)
    

ToDo: run this against production db

In [37]:
for x in tqdm(doc.any_xpath(".//tei:bibl[@xml:id][./tei:listBibl]")):
    ref = []
    for y in x.xpath(".//tei:bibl[./tei:biblScope]", namespaces=nsmap):
        volume = y.xpath("./tei:biblScope[@unit='volume']", namespaces=nsmap)[0].text
        page = y.xpath("./tei:biblScope[@unit='page']", namespaces=nsmap)[0].text
        if volume.endswith("."):
            volume = normalize_string(volume[:-1])
        try:
            page = f"S. {normalize_string(page)}"
        except AttributeError:
            continue
        quote = f"{volume}, {page}."
        ref.append(quote)
    all_quotes = (" *** ".join(ref))
    if len(quote) > 2:
        more.append(all_quotes)     
        
    hbtv_uri = get_xmlid(x)
    work_uri = f"{bahr_url}{hbtv_uri}"
    work = Uri.objects.get(uri=work_uri).entity.get_child_entity()
    orig_ref = work.references
    if orig_ref:
        if all_quotes in orig_ref:
            continue
    try:
        new_ref = orig_ref + "\n" + all_quotes
    except TypeError:
        new_ref = "\n" + all_quotes
    work.references = new_ref
    work.save()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2148/2148 [00:27<00:00, 79.08it/s]
